In [2]:
import numpy as np
import pandas as pd
import re 
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from nltk.corpus import stopwords
from scipy import optimize
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import PorterStemmer
import string
from scipy.io import loadmat
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from utils import data_splitter,text_process,build_freqs

In [3]:

X_train, X_test, y_train, y_test,train_lab,test_lab = data_splitter('resources/dataset.csv')


In [4]:
freqs = build_freqs(X_train, train_lab)

In [8]:
def sigmoid(z):
    h = 1 / (1 + np.exp(-z))
    return h

def gradientDescent(x, y, theta, alpha, num_iters):
    m = x.shape[0]
    for i in range(0, num_iters):
    
        z = np.dot(x,theta)
        h = sigmoid(z)
        J = -1./m * (np.dot(y.transpose(), np.log(h)) + np.dot((1-y).transpose(),np.log(1-h)))    

        theta = theta - (alpha/m) * np.dot(x.transpose(),(h-y))
        
  
    J = float(J)
    return J, theta


def extract_features(text, freqs):
    x = np.zeros((1, 3))
    x[0,0] = 1
    word_l = text_process(text)
    for word in word_l:
        x[0,1] += freqs.get((word, 1.0),0)
        x[0,2] += freqs.get((word, 0.0),0)
    assert(x.shape == (1, 3))
    return x

## Training 

In [9]:
X = np.zeros((len(X_train), 3))
for i in range(len(X_train)):
    X[i, :]= extract_features(X_train[i], freqs)
J, theta = gradientDescent(X, y_train, np.zeros((3, 1)), 1e-9, 1500)

print(f"The cost after training is {J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")

The cost after training is 0.49099771.
The resulting vector of weights is [-6e-08, 0.00031954, -0.00038018]


In [10]:
def predict_text(text, freqs, theta):
    x = extract_features(text,freqs)
    y_pred = sigmoid(np.dot(x,theta))
    return y_pred

In [12]:
def test_logistic_regression(test_x, test_y, freqs, theta):
    y_hat = []
    for text in test_x:
        y_pred = predict_text(text, freqs, theta)
        if y_pred > 0.5:
            y_hat.append(1)
        else:
            y_hat.append(0)
            
    
    return y_hat

In [13]:
y_hat = test_logistic_regression(X_test, y_test, freqs, theta)
accuracy = (y_hat==np.squeeze(y_test)).sum()/len(X_test)
print(f"Logistic regression model's accuracy = {accuracy:.4f}")

Logistic regression model's accuracy = 0.8238


In [14]:
print(classification_report(y_test,y_hat))

              precision    recall  f1-score   support

         0.0       0.84      0.80      0.82     20000
         1.0       0.81      0.85      0.83     20000

    accuracy                           0.82     40000
   macro avg       0.82      0.82      0.82     40000
weighted avg       0.82      0.82      0.82     40000



In [15]:
my_text ="What do you call a bee that can’t make up its mind? A maybe."
y_hat = predict_text(my_text, freqs, theta)
print(y_hat)
if y_hat > 0.5:
    print('Humor text')
else: 
    print('Non_Humor text')

[[0.89242283]]
Humor text


In [16]:
my_text ="I miss you"
y_hat = predict_text(my_text, freqs, theta)
print(y_hat)
if y_hat > 0.5:
    print('Humor text')
else: 
    print('Non_Humor text')

[[0.4919507]]
Non_Humor text


## Using Sklearn

In [43]:
df = pd.read_csv('resources/dataset.csv')
X_train, X_test, y_train, y_test=train_test_split(df['text'], df['humor'], test_size=0.2)

In [44]:
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)), 
    ('tfidf', TfidfTransformer()), 
('classifier', GridSearchCV(LogisticRegression(),param_grid={'C': np.logspace(-3,3,6),'max_iter':[10,100,500], "penalty":["l1","l2"]} ,refit=True,verbose=3))
])

In [45]:
rf_model=pipeline.fit(X_train,y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 1/5] END ....C=0.001, max_iter=10, penalty=l1;, score=nan total time=   0.0s
[CV 2/5] END ....C=0.001, max_iter=10, penalty=l1;, score=nan total time=   0.0s
[CV 3/5] END ....C=0.001, max_iter=10, penalty=l1;, score=nan total time=   0.0s
[CV 4/5] END ....C=0.001, max_iter=10, penalty=l1;, score=nan total time=   0.0s
[CV 5/5] END ....C=0.001, max_iter=10, penalty=l1;, score=nan total time=   0.0s
[CV 1/5] END ..C=0.001, max_iter=10, penalty=l2;, score=0.828 total time=   0.1s
[CV 2/5] END ..C=0.001, max_iter=10, penalty=l2;, score=0.829 total time=   0.1s
[CV 3/5] END ..C=0.001, max_iter=10, penalty=l2;, score=0.826 total time=   0.2s
[CV 4/5] END ..C=0.001, max_iter=10, penalty=l2;, score=0.828 total time=   0.2s
[CV 5/5] END ..C=0.001, max_iter=10, penalty=l2;, score=0.826 total time=   0.1s
[CV 1/5] END ...C=0.001, max_iter=100, penalty=l1;, score=nan total time=   0.0s
[CV 2/5] END ...C=0.001, max_iter=100, penalty=

In [46]:
predictions = pipeline.predict(X_test)

In [51]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

       False       0.90      0.91      0.91     19897
        True       0.91      0.90      0.91     20103

    accuracy                           0.91     40000
   macro avg       0.91      0.91      0.91     40000
weighted avg       0.91      0.91      0.91     40000



In [52]:
print(confusion_matrix(y_test,predictions))

[[18079  1818]
 [ 1953 18150]]
